## Mounting Drive


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My\ Drive/Projects_repo/face_emotion_recog/

# Installing and Importing the required libraries 

In [ ]:
!pip install -q torchbearer
!pip install livelossplot

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
%matplotlib inline

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot import PlotLossesKerasTF
import tensorflow as tf
import cv2
from keras import models

print(tf.__version__)

# Training and Testing Batches

In [ ]:
img_size = 48
batch_size = 64

data_train = ImageDataGenerator(horizontal_flip=True)
train_gen = data_train.flow_from_directory("##PATH TO THE TRAINING IMAGES DIRECTORY",
                                           target_size=(img_size, img_size),
                                           color_mode='grayscale',
                                           batch_size=batch_size,
                                           class_mode='categorical',
                                           shuffle=True)

data_test = ImageDataGenerator(horizontal_flip=True)
test_gen = data_test.flow_from_directory("##PATH TO THE TEST IMAGES DIRECTORY##",
                                           target_size=(img_size, img_size),
                                           color_mode='grayscale',
                                           batch_size=batch_size,
                                           class_mode='categorical',
                                           shuffle=True)

# Convolution Neural Network for the Macro Expressions Detector

In [ ]:
model = Sequential()

#Convolution Layer-1
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Convolution Layer-2
model.add(Conv2D(128, (5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Convolution Layer-3
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#Convolution Layer-4
model.add(Conv2D(512, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25) )

model.add(Dense(7,activation='softmax'))

opt = Adam(learning_rate=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


# Training and Testing the model

In [ ]:
epochs = 20
steps_per_epoch = train_gen.n//train_gen.batch_size
test_steps = test_gen.n//test_gen.batch_size

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=2, min_lr=0.00001, mode='auto')

checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)

callbacks = [PlotLossesKerasTF(), checkpoint, reduce_lr]

history = model.fit(
    x=train_gen,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data = test_gen,
    validation_steps = test_steps,
    callbacks=callbacks
)

## Saving the Model as JSON

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file_write(model_json)
model.save_weights("##Path to save your model_weights.h5 file##")
print("Model Saved Succesfully")